## Notebook for adding a train or test column to rows in a dataframe of wells loaded but not features added yet

In [18]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import welly
from welly import Well
import lasio
import glob
import pickle
import math
import dask
import dask.dataframe as dd
from dask.distributed import Client
import random 

In [17]:
print(welly.__version__)
print(dask.__version__)
print(pd.__version__)

0.3.5
0.17.5
0.23.0


In [7]:
%%timeit
import os
env = %env

73.5 µs ± 3.05 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
#### Had to change display options to get this to print in full!
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 100000

In [9]:
knn_dir = "../WellsKNN/"
load_dir = "../loadLAS/"
features_dir = "../createFeatures/"
check_dir = "../CheckAvailableData/"
loadLas_dir = "../loadLAS/"


## File with wells loaded but not features or train test split yet

In [10]:
file_with_wells = 'df_all_wells_noKNNorFeatures_20180927_a.h5'

## File to write to at end = 

In [32]:
h5_filename_df_wells_wTrainTestSplitCol = 'df_all_wells_noKNNorFeatures_wTrainSplit_20180927_c.h5'

## Split variables

In [33]:
split_variable = 0.8

## Import all the data

In [34]:
##df_1.to_hdf('df_all_wells_noKNNorFeatures_20180920_a.h5', key='df', mode='w')
df_all_Col_preSplit = pd.read_hdf(load_dir+file_with_wells, 'df')

## Split data into train & test on a well by well basis, not row basis

In [35]:
UWIs = list(df_all_Col_preSplit['UWI'].unique())

In [36]:
numberOfTrainingWells = math.floor(len(UWIs)*split_variable)
numberOfTrainingWells

1280

Randomly select that number of UWIs for training and the ones left for test

In [37]:
UWIs_training = random.sample(UWIs, numberOfTrainingWells)

In [38]:
UWIs_test = [x for x in UWIs if x not in UWIs_training]

In [39]:
print("train",len(UWIs_training))
print("test",len(UWIs_test))

train 1280
test 321


In [40]:
df_all_Col_preSplit_wTrainTest = df_all_Col_preSplit.copy()

In [41]:
df_all_Col_preSplit_wTrainTest['trainOrTest'] = np.where(df_all_Col_preSplit_wTrainTest['UWI'].isin(UWIs_training), 'train', 'test')

In [42]:
df_all_Col_preSplit_wTrainTest.tail()

,CALI,COND,DELT,DEPT,DPHI,DT,GR,ILD,ILM,NPHI,PHID,RHOB,SFL,SFLU,SN,SP,UWI,trainOrTest
959,243.882,NaN,NaN,764.006,0.136,NaN,26.517,4.595,NaN,0.468,NaN,NaN,NaN,NaN,NaN,NaN,00-07-06-077-23W4-0.LAS,train
960,243.751,NaN,NaN,764.256,0.124,NaN,33.226,4.490,NaN,0.460,NaN,NaN,NaN,NaN,NaN,NaN,00-07-06-077-23W4-0.LAS,train
961,244.046,NaN,NaN,764.506,0.104,NaN,2.654,4.395,NaN,0.449,NaN,NaN,NaN,NaN,NaN,NaN,00-07-06-077-23W4-0.LAS,train
962,244.763,NaN,NaN,764.756,0.072,NaN,7.806,4.327,NaN,0.453,NaN,NaN,NaN,NaN,NaN,NaN,00-07-06-077-23W4-0.LAS,train
963,245.368,NaN,NaN,765.006,0.056,NaN,11.962,4.284,NaN,0.462,NaN,NaN,NaN,NaN,NaN,NaN,00-07-06-077-23W4-0.LAS,train


## Save to file

In [43]:
df_all_Col_preSplit_wTrainTest.to_hdf(h5_filename_df_wells_wTrainTestSplitCol, key='df', mode='w')